In [1]:
import pandas as pd
PATH = 'data_final_for_dls.jsonl'
PATH_NEW = 'data_final_for_dls_new_new.jsonl'
data = pd.read_json(path_or_buf=PATH_NEW, lines=True)

In [2]:
data['relevance'].value_counts()

relevance
1.0    15882
0.0    14509
0.1     4703
Name: count, dtype: int64

In [3]:
train_data = data[570:]
eval_data = data[:570]
eval_data = eval_data[eval_data["relevance"] != 0.1]
eval_data.head()

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,relevance_new
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите...",1.0
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан...",0.0
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос...",1.0
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...,1.0
5,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,Быстрое питание,1219173871,Rostic's предлагает различные наборы быстрого ...,1.0,"Организация занимается быстрым питанием, предо...",1.0


In [8]:
from openai import OpenAI
import os

API_KEY = "ollama" # "ollama" #"QAFlNVFC15GT4nraEY8ZuQduYLeutJZG"
BASE_URL = "http://localhost:11434/v1/" # "http://localhost:11434/v1/" # "https://api.mistral.ai/v1"
MODEL = "llama3.2:3b" # "mistral-medium" # "mistral-large-2411" #"llama3.2:3b" # "mistral-small-latest"

class RelevanceLLMAgent:
    def __init__(self, model=MODEL, temperature=0.0):
        self.client = OpenAI(base_url=BASE_URL, api_key=API_KEY) # os.environ["OPENAI_API_KEY"])
        self.model = model
        self.temperature = temperature

    def predict(self, query: str, org: dict) -> bool:
        system_prompt = (
            "Ты помощник, оценивающий релевантность информации об организации запросу пользователя."
            "Продумай ответ шаг за шагом, приняв во внимание информацию из контекста."
            "Отвечай только 'релевантно' или 'не релевантно'."
        )

        user_prompt = (
            f"Запрос пользователя: {query}\n\n"
            f"Название: {org.get('name', '')}\n"
            f"Категория: {org.get('normalized_main_rubric_name_ru', '')}\n"
            f"Отзывы: {org.get('reviews_summarized', '')}\n"
            f"Прайс: {org.get('prices_summarized', '')}\n"
            f"Адрес: {org.get('address', '')}\n\n"
            "Ответь: релевантно или не релевантно?"
        )

        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )

        answer = response.choices[0].message.content.strip().lower()
        return "релевантно" in answer, answer


In [9]:
agent = RelevanceLLMAgent()

query = "сигары"
org = {
    "name": "Tabaccos",
    "normalized_main_rubric_name_ru": "Магазин табака и курительных принадлежностей",
    "reviews_summarized": "Организация занимается продажей табака, курительных изделий и аксессуаров.",
    "prices_summarized": None,
    "address": "Москва, Дубравная улица, 34/29"
}

agent.predict(query, org)


InternalServerError: Error code: 502

## Классификатор

In [70]:
train_data = train_data.fillna("")
train_data["relevance"] = train_data["relevance"].astype(int)

In [71]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(train_data, test_size=573, random_state=42)

In [72]:
df_val = df_val[df_val["relevance"] != 0.1]
df_val.shape

(573, 9)

In [59]:
X_train = df_train.drop("relevance", axis= 1)
X_val = df_val.drop("relevance", axis= 1)
y_train = df_train.relevance
y_val = df_val.relevance

In [60]:
X_train.head()

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,reviews_summarized
23647,туры на новый год 2018 куда поехать недорого,"Москва, 1-я Тверская-Ямская улица, 25, стр. 1",Русь; Rus',Туроператор,1193822323,,
20019,разливное пиво 24 часа,"Москва, улица Сретенка, 1с1",Азбука daily; Azbuka daily; Азбука Вкуса; Азбу...,Магазин продуктов,41730131149,,Организация «Азбука daily» занимается продажей...
15740,чиносы мужские,"Москва, Кировоградская улица, 13А",Kanzler; KANZLER; Kanzler - магазин мужской од...,Магазин одежды,234359465448,Kanzler предлагает широкий ассортимент брюк из...,"Организация занимается продажей одежды, имеет ..."
29149,киа официальный сайт,"Москва, Коптевская улица, 69А",Kia Favorit Motors Север; Kia Favorit Motors N...,Автосалон,169217886341,Автосалон Kia Favorit Motors Север предлагает ...,Организация занимается продажей автомобилей Ki...
28780,достало ресторан,"Московская область, Ленинский городской округ,...",Завидное; ZaVidnoe; ЗаВидное,Ресторан,228176463096,Ресторан «Завидное» предлагает широкий ассорти...,Организация занимается ресторанным бизнесом и ...


In [61]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33951 entries, 23647 to 16365
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Text                            33951 non-null  object
 1   address                         33951 non-null  object
 2   name                            33951 non-null  object
 3   normalized_main_rubric_name_ru  33951 non-null  object
 4   permalink                       33951 non-null  int64 
 5   prices_summarized               33951 non-null  object
 6   reviews_summarized              33951 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.1+ MB


In [62]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report

text_features = ["Text", "name", 'address', "normalized_main_rubric_name_ru", "reviews_summarized", "prices_summarized"]

# Обучающая и валидационная выборки
X_train = X_train[text_features]
X_val = X_val[text_features]

# # Создаем CatBoost Pool
# train_pool = Pool(X_train, label=y_train, text_features=text_features)
# val_pool = Pool(X_val, label=y_val, text_features=text_features)

# Обучение
model = CatBoostClassifier(
    text_features=text_features,
    iterations=500,
    learning_rate=0.1,
    depth=6,
    eval_metric='TotalF1',
    verbose=50,
    task_type="CPU"  # или GPU если доступно
)

model.fit(X_train, y_train, early_stopping_rounds=50)


0:	learn: 0.6663421	total: 233ms	remaining: 1m 56s
50:	learn: 0.6820354	total: 11.8s	remaining: 1m 43s
100:	learn: 0.6893578	total: 28s	remaining: 1m 50s
150:	learn: 0.7017189	total: 45.4s	remaining: 1m 44s
200:	learn: 0.7128113	total: 1m 2s	remaining: 1m 33s
250:	learn: 0.7227497	total: 1m 19s	remaining: 1m 19s
300:	learn: 0.7338967	total: 1m 38s	remaining: 1m 4s
350:	learn: 0.7430842	total: 1m 55s	remaining: 49.1s
400:	learn: 0.7509937	total: 2m 14s	remaining: 33.1s
450:	learn: 0.7595784	total: 2m 30s	remaining: 16.4s
499:	learn: 0.7686705	total: 2m 47s	remaining: 0us


In [63]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

y_pred = model.predict(X_val)
y_proba = model.predict_proba(X_val)[:, 1]

print("Accuracy:", accuracy_score(y_val, y_pred))
print("F1-score:", f1_score(y_val, y_pred))
print("ROC-AUC:", roc_auc_score(y_val, y_proba))

Accuracy: 0.6858638743455497
F1-score: 0.6442687747035574
ROC-AUC: 0.7574556343732358


In [73]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm

def evaluate_llm(agent, df, max_samples=100):
    y_true, y_pred = [], []

    for _, row in tqdm(df.head(max_samples).iterrows(), total=min(len(df), max_samples)):
        org = {
            "name": row["name"],
            "normalized_main_rubric_name_ru": row["normalized_main_rubric_name_ru"],
            "reviews_summarized": row["reviews_summarized"],
            "prices_summarized": row["prices_summarized"],
            "address": row.get("address", ""),
        }
        pred = agent.predict(row["Text"], org)
        y_true.append(row["relevance"])
        y_pred.append(int(pred))

    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
    }


In [112]:
import time

def evaluate_llm(agent, df, max_samples=100, min_interval_sec=2):
    y_true, y_pred = [], []
    last_call_time = None

    for _, row in tqdm(df.head(max_samples).iterrows(), total=min(len(df), max_samples)):
        start_time = time.time()

        org = {
            "name": row.get("name", ""),
            "normalized_main_rubric_name_ru": row.get("normalized_main_rubric_name_ru", ""),
            "reviews_summarized": row.get("reviews_summarized", ""),
            "prices_summarized": row.get("prices_summarized", ""),
            "address": row.get("address", ""),
        }

        try:
            pred = agent.predict(row["Text"], org)
            y_pred.append(int(pred))
            y_true.append(row["relevance"])
        except Exception as e:
            print("⚠️ Ошибка:", e)
            continue

        # Ждём остаток до полного интервала
        elapsed = time.time() - start_time
        wait_time = max(0.0, min_interval_sec - elapsed)
        time.sleep(wait_time)

    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
    }


In [ ]:
%%time
# Ollama metrix
metrics = evaluate_llm(agent, df_val, max_samples=500)
print(metrics)

In [113]:
%%time
# Mistral metrix
metrics = evaluate_llm(agent, df_val, max_samples=100)
print(metrics)

100%|██████████| 100/100 [03:22<00:00,  2.03s/it]

{'accuracy': 0.51, 'f1': 0.6754966887417219, 'roc_auc': 0.5}
Wall time: 3min 22s
